# Genetic Algorithm for Neural Network Optimization

---

DQN and PPO are deep reinforcement learning methods that optimize neural network weights through gradient descent, computing how each weight should change to improve performance, then applying small, calculated adjustments through backpropagation. This process requires differentiable loss functions, careful learning rate tuning, and mechanisms like experience replay or policy clipping to remain stable.

A genetic algorithm (GA) approaches the same reinforcement learning problem from a fundamentally different angle. It treats the neural network's weights as a flat array of numbers, called a "genome", and applies principles borrowed from biological evolution to find good values, without ever computing a gradient. While still reinforcement learning (the agent learns from environment interaction and reward signals), it is not deep learning, as the optimization is evolutionary rather than gradient-based.

The process works as follows:

1. **Initialization**: Create a population of 50 random genomes, each representing a complete neural network with random weights.

2. **Evaluation**: Each genome is decoded into a neural network and tested directly in the LunarLander environment. The network receives the same 8 observations (position, velocity, angle, leg contacts) and outputs one of 4 actions, identical to the DQN/PPO networks. Its fitness score is simply the total reward accumulated during the episode.

3. **Selection**: Genomes are ranked by fitness. The top 20% become eligible parents, and the top 3 survive unchanged into the next generation (elitism).

4. **Crossover**: Two parents are selected and their weights are combined to produce a child genome, mixing genetic material from both successful individuals.

5. **Mutation**: Small random perturbations are applied to the child's weights, introducing variation. The mutation rate decays linearly over generations, with large perturbations early for exploration and smaller ones later for refinement.

6. **Repeat**: The new population replaces the old one, and the process repeats for thousands of generations.

The key distinction is what drives learning. In DQN/PPO, the network receives precise mathematical feedback about which direction to adjust each weight. In the GA, there is no such signal. The algorithm only knows "this set of weights scored 280, that one scored 150." It discovers good weights purely through trial, selection, and incremental refinement over generations.

This makes the GA less sample-efficient, requiring roughly 150x more environment interactions than DQN or PPO to reach comparable performance, but also entirely gradient-free. It requires no loss function design, no replay buffer, no policy clipping, and no learning rate schedule. The neural network architecture itself is a simple feedforward network with two hidden layers of 10 neurons each using tanh activations, far smaller than the 256x256 networks used by DQN and PPO, yet sufficient for the task.

In [ ]:
import os, sys, random, time, glob
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

import gymnasium as gym
import imageio.v2 as imageio
from IPython.display import Image, display

In [ ]:
from genetic.evolution.genetic_algorithm import GeneticAlgorithm
from genetic.evolution.neural_network import NeuralNetwork

In [ ]:
# Global Configuration

SEED_LIST = [42, 123, 3407]

NOTEBOOK_DIR = os.path.dirname(os.path.abspath("__file__"))
GYMNASIUM_MODEL = "LunarLander-v3"

WIND_ENABLED = False

# Session prefix — used in final genome filenames (e.g. annex1_ga_42.npy)
SESSION_PREFIX = "annex1"

# GA hyperparameters
INPUT_SIZE = 8
HIDDEN1_SIZE = 10
HIDDEN2_SIZE = 10
OUTPUT_SIZE = 4

POPULATION_SIZE = 50
MUTATION_RATE = 0.05
GENERATIONS = 5000
EVAL_SEEDS_PER_GEN = 3  # seeds used to evaluate each genome during training

# Evaluation
EVALUATION_EPISODES = 20

# Trajectory visualization episodes
TRAJECTORY_EPISODES = 3

# Parallelization
MAX_WORKERS = 20

# LunarLander-v3 action labels
ACTION_LABELS = ["Do Nothing", "Fire Left", "Fire Main", "Fire Right"]

print(f"Session prefix: {SESSION_PREFIX}")
print(f"Seeds: {SEED_LIST}")
print(f"Wind enabled: {WIND_ENABLED}")
print(f"Generations: {GENERATIONS}")
print(f"Population size: {POPULATION_SIZE}")
print(f"Mutation rate: {MUTATION_RATE}")
print(f"Eval seeds per generation: {EVAL_SEEDS_PER_GEN}")
print(f"Evaluation episodes per seed: {EVALUATION_EPISODES}")
print(f"Max workers: {MAX_WORKERS}")

In [ ]:
print("Python:", sys.version.split()[0])
print("NumPy:", np.__version__)

In [ ]:
# Environment inspection
env_tmp = gym.make(GYMNASIUM_MODEL)

print("Observation space:", env_tmp.observation_space)
print("Action space:", env_tmp.action_space)

obs, info = env_tmp.reset()
print("Initial observation:", obs)

env_tmp.close()

In [ ]:
def set_all_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)


def evaluate_genome_deterministic(genome, n_episodes, seed=None):
    """
    Evaluate a genome deterministically over n_episodes.
    No early termination. Returns list of per-episode rewards.
    """
    nn = NeuralNetwork(INPUT_SIZE, HIDDEN1_SIZE, HIDDEN2_SIZE, OUTPUT_SIZE, genome)
    env = gym.make(GYMNASIUM_MODEL, enable_wind=WIND_ENABLED)
    rewards = []

    for ep in range(n_episodes):
        ep_seed = seed + ep if seed is not None else None
        obs, _ = env.reset(seed=ep_seed)
        total_reward = 0.0
        done = False

        while not done:
            output = nn.forward(obs)
            action = np.argmax(output)
            obs, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            total_reward += float(reward)

        rewards.append(total_reward)

    env.close()
    return rewards


def record_genome_gif(genome, seed, output_path):
    """
    Record a single episode GIF of a genome.
    """
    nn = NeuralNetwork(INPUT_SIZE, HIDDEN1_SIZE, HIDDEN2_SIZE, OUTPUT_SIZE, genome)
    env = gym.make(GYMNASIUM_MODEL, render_mode="rgb_array", enable_wind=WIND_ENABLED)
    obs, _ = env.reset(seed=seed)
    frames = []
    done = False

    while not done:
        frames.append(env.render())
        output = nn.forward(obs)
        action = np.argmax(output)
        obs, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

    env.close()
    imageio.mimsave(output_path, frames, fps=30, loop=0)
    return output_path

In [ ]:
# Training loop: GA x seeds

from IPython.display import display as ipy_display

training_results = {}   # {seed: {"best": [], "avg": [], "worst": []}}
training_times = {}     # {seed: seconds}
best_genomes = {}       # {seed: genome}
best_genome_paths = {}  # {seed: path}
total_env_steps = {}    # {seed: int}
eval_histories = {}     # {seed: list of eval dicts}

CHART_UPDATE_FREQ = 100
EVAL_FREQ_GENS = 100
EVAL_N_EPISODES = 20
SOLVED_THRESHOLD = 200

for seed in SEED_LIST:
    print(f"\n{'='*60}")
    print(f"GA | Seed {seed}")
    print(f"{'='*60}\n")

    # Create timestamped run directory
    run_timestamp = datetime.now().strftime("%Y-%m-%d_%H_%M_%S")
    run_dir = os.path.join(NOTEBOOK_DIR, "../models", "ga", run_timestamp)
    os.makedirs(run_dir, exist_ok=True)

    set_all_seeds(seed)

    ga = GeneticAlgorithm(
        INPUT_SIZE, HIDDEN1_SIZE, HIDDEN2_SIZE, OUTPUT_SIZE,
        POPULATION_SIZE, MUTATION_RATE, render=False, max_workers=MAX_WORKERS,
        generations=GENERATIONS
    )

    best_fitness_history = []
    avg_fitness_history = []
    worst_fitness_history = []
    env_steps = 0
    sorted_population = []

    # Best model tracking (matches DQN/PPO two-tier selection)
    best_combined_score = -np.inf
    best_genome = np.array([])
    best_eval_mean = 0.0
    best_eval_std = np.inf
    best_eval_success = 0.0
    best_gen = 0
    any_solved = False
    eval_history = []

    plot_handle = None
    stats_handle = None
    eval_handle = None

    t_start = time.time()

    for gen in range(GENERATIONS):
        sorted_population = ga.evaluate_population(gen)
        fitness_values = [fitness for _, fitness in sorted_population]
        best_fitness = fitness_values[0]
        worst_fitness = fitness_values[-1]
        avg_fitness = sum(fitness_values) / len(fitness_values)

        best_fitness_history.append(best_fitness)
        avg_fitness_history.append(avg_fitness)
        worst_fitness_history.append(worst_fitness)

        env_steps += POPULATION_SIZE * EVAL_SEEDS_PER_GEN * 300

        # Periodic deterministic evaluation of generation's best genome
        if (gen + 1) % EVAL_FREQ_GENS == 0:
            candidate_genome = sorted_population[0][0]
            eval_rewards = evaluate_genome_deterministic(
                candidate_genome, EVAL_N_EPISODES, seed=seed
            )
            eval_mean = np.mean(eval_rewards)
            eval_std = np.std(eval_rewards)
            eval_success = np.sum(np.array(eval_rewards) >= SOLVED_THRESHOLD) / len(eval_rewards) * 100
            combined_score = eval_mean - eval_std
            is_solved = eval_mean >= SOLVED_THRESHOLD

            eval_history.append({
                "generation": gen + 1,
                "mean": eval_mean,
                "std": eval_std,
                "success": eval_success,
                "score": combined_score,
                "solved": is_solved,
            })

            # Two-tier best model selection (matches DQN/PPO logic)
            save_new_best = False
            if is_solved:
                if not any_solved:
                    save_new_best = True
                    any_solved = True
                elif combined_score > best_combined_score:
                    save_new_best = True
            elif not any_solved:
                if combined_score > best_combined_score:
                    save_new_best = True

            solved_tag = " [SOLVED]" if is_solved else ""
            eval_text = (
                f"Eval @ Gen {gen + 1} | "
                f"Reward: {eval_mean:.2f} +/- {eval_std:.2f}{solved_tag} | "
                f"Success: {eval_success:.0f}% | "
                f"Score (mean-std): {combined_score:.2f} | "
                f"Best: {best_combined_score:.2f}"
            )

            if save_new_best:
                eval_text += " >> New best genome!"
                best_combined_score = combined_score
                best_eval_mean = eval_mean
                best_eval_std = eval_std
                best_eval_success = eval_success
                best_gen = gen + 1
                best_genome = candidate_genome.copy()
                np.save(os.path.join(run_dir, "best_genome.npy"), best_genome)

            if eval_handle is None:
                eval_handle = ipy_display(eval_text, display_id=True)
            else:
                eval_handle.update(eval_text)

        if (gen + 1) % CHART_UPDATE_FREQ == 0:
            recent_avg = np.mean(avg_fitness_history[-100:])
            stats_text = (
                f"Generation {gen + 1} | "
                f"Best: {best_fitness:>8.2f} | "
                f"Avg: {avg_fitness:>8.2f} | "
                f"Worst: {worst_fitness:>8.2f} | "
                f"Best Score: {best_combined_score:>8.2f} | "
                f"Recent Avg(100): {recent_avg:.1f} | "
                f"MR: {ga.mutation_rate:.4f}"
            )
            if stats_handle is None:
                stats_handle = ipy_display(stats_text, display_id=True)
            else:
                stats_handle.update(stats_text)

            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))

            ax1.plot(best_fitness_history, alpha=0.3, color='blue')
            ax1.plot(avg_fitness_history, alpha=0.3, color='orange')
            window = min(50, len(best_fitness_history))
            rolling_best = pd.Series(best_fitness_history).rolling(window).mean()
            rolling_avg = pd.Series(avg_fitness_history).rolling(window).mean()
            ax1.plot(rolling_best, color='blue', linewidth=2, label='Best (rolling)')
            ax1.plot(rolling_avg, color='orange', linewidth=2, label='Avg (rolling)')
            ax1.axhline(y=200, color='red', linestyle='--', alpha=0.5)
            ax1.set_title(f"Fitness - Gen {gen + 1}")
            ax1.set_xlabel("Generation")
            ax1.set_ylabel("Fitness")
            ax1.legend(fontsize=8)
            ax1.grid(True, alpha=0.3)

            success = np.array(best_fitness_history) >= 200
            rolling_success = pd.Series(success.astype(float)).rolling(window).mean() * 100
            ax2.plot(rolling_success, color='green', linewidth=2)
            ax2.axhline(y=100, color='green', linestyle=':', alpha=0.4)
            ax2.set_title(f"Success Rate (best >= 200)")
            ax2.set_xlabel("Generation")
            ax2.set_ylabel("Success Rate (%)")
            ax2.set_ylim(-5, 105)
            ax2.grid(True, alpha=0.3)

            plt.tight_layout()

            if plot_handle is None:
                plot_handle = ipy_display(fig, display_id=True)
            else:
                plot_handle.update(fig)
            plt.close(fig)

        ga.next_generation(sorted_population)

    t_elapsed = time.time() - t_start

    # Save final genome (best of last generation)
    final_genome = sorted_population[0][0]
    final_path = os.path.join(run_dir, f"{SESSION_PREFIX}_ga_{seed}.npy")
    np.save(final_path, final_genome)

    # Save eval log
    eval_log_dir = os.path.join(run_dir, "eval_log")
    os.makedirs(eval_log_dir, exist_ok=True)
    if eval_history:
        np.savez(
            os.path.join(eval_log_dir, "evaluations"),
            generations=np.array([e["generation"] for e in eval_history]),
            means=np.array([e["mean"] for e in eval_history]),
            stds=np.array([e["std"] for e in eval_history]),
            scores=np.array([e["score"] for e in eval_history]),
            success_rates=np.array([e["success"] for e in eval_history]),
        )

    # Save fitness history
    history_path = os.path.join(run_dir, f"fitness_history_seed{seed}.npz")
    np.savez(history_path,
             best=best_fitness_history,
             avg=avg_fitness_history,
             worst=worst_fitness_history)

    training_results[seed] = {
        "best": best_fitness_history,
        "avg": avg_fitness_history,
        "worst": worst_fitness_history,
    }
    training_times[seed] = t_elapsed
    best_genomes[seed] = best_genome
    best_genome_paths[seed] = {
        "run_dir": run_dir,
        "final": final_path,
        "best": os.path.join(run_dir, "best_genome.npy"),
    }
    total_env_steps[seed] = env_steps
    eval_histories[seed] = eval_history

    print(f"\nTraining time: {t_elapsed/60:.1f} min ({t_elapsed:.0f} s)")
    print(f"Final genome:  {final_path}")
    print(f"Best genome:   {os.path.join(run_dir, 'best_genome.npy')}")
    print(f"Eval log:      {eval_log_dir}")
    print(
        f"Best genome stats: "
        f"Reward: {best_eval_mean:.2f} +/- {best_eval_std:.2f} | "
        f"Success: {best_eval_success:.0f}% | "
        f"Score (mean-std): {best_combined_score:.2f} | "
        f"@ Gen {best_gen}"
    )
    print(f"Estimated env steps: {env_steps:,}")

print(f"\nGA: All {len(SEED_LIST)} seeds trained.")

# Training Summary Table
print(f"\n{'='*60}")
print("BEST MODEL SUMMARY (all seeds)")
print(f"{'='*60}\n")

rows = []
for seed in SEED_LIST:
    best_eval = max(eval_histories[seed], key=lambda x: x["score"])
    rows.append({
        "Seed": seed,
        "Mean Reward": f"{best_eval['mean']:.2f}",
        "Std Reward": f"{best_eval['std']:.2f}",
        "Success": f"{best_eval['success']:.0f}%",
        "Score (mean-std)": f"{best_eval['score']:.2f}",
        "@ Generation": best_eval["generation"],
        "Time (min)": f"{training_times[seed]/60:.1f}",
    })

print(pd.DataFrame(rows).to_string(index=False))

In [ ]:
# Training Time Summary

rows = []
for seed in SEED_LIST:
    t = training_times[seed]
    rows.append({
        "Algorithm": "GA",
        "Seed": seed,
        "Time (s)": f"{t:.0f}",
        "Time (min)": f"{t/60:.1f}",
    })

times = list(training_times.values())
rows.append({
    "Algorithm": "GA",
    "Seed": "Mean",
    "Time (s)": f"{np.mean(times):.0f}",
    "Time (min)": f"{np.mean(times)/60:.1f}",
})

print("*** TRAINING TIME SUMMARY ***")
print(f"Generations: {GENERATIONS} | Population: {POPULATION_SIZE} | Workers: {MAX_WORKERS}")
print()
print(pd.DataFrame(rows).to_string(index=False))

In [ ]:
# Per-Seed: Fitness over Generations

seed_colors = list(plt.colormaps["tab10"](range(10)))

fig, axes = plt.subplots(1, len(SEED_LIST), figsize=(6 * len(SEED_LIST), 5), sharey=True)
if len(SEED_LIST) == 1:
    axes = [axes]

for ax, seed in zip(axes, SEED_LIST):
    ax.plot(training_results[seed]["best"], alpha=0.7, label="Best")
    ax.plot(training_results[seed]["avg"], alpha=0.7, label="Average")
    ax.plot(training_results[seed]["worst"], alpha=0.5, label="Worst")
    ax.axhline(y=200, color='red', linestyle='--', alpha=0.5)
    ax.set_title(f"Seed {seed}")
    ax.set_xlabel("Generation")
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=8)

axes[0].set_ylabel("Fitness")
fig.suptitle("GA - Fitness over Generations", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Aggregated: Rolling Best Fitness Overlay (all seeds on one chart)

plt.figure(figsize=(14, 6))
for i, seed in enumerate(SEED_LIST):
    rolling = pd.Series(training_results[seed]["best"]).rolling(50).mean()
    plt.plot(rolling, color=seed_colors[i], linewidth=2, label=f"Seed {seed}")

plt.axhline(y=200, color='red', linestyle='--', label='Solved Threshold (200)')
plt.title("GA Training: Rolling Best Fitness (window=50)", fontsize=14)
plt.xlabel("Generation")
plt.ylabel("Fitness")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Aggregated: Rolling Average Fitness Overlay (all seeds on one chart)

plt.figure(figsize=(14, 6))
for i, seed in enumerate(SEED_LIST):
    rolling = pd.Series(training_results[seed]["avg"]).rolling(50).mean()
    plt.plot(rolling, color=seed_colors[i], linewidth=2, label=f"Seed {seed}")

plt.axhline(y=200, color='red', linestyle='--', label='Solved Threshold (200)')
plt.title("GA Training: Rolling Average Fitness (window=50)", fontsize=14)
plt.xlabel("Generation")
plt.ylabel("Fitness")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Rolling Success Rate over Training (best fitness >= 200)

plt.figure(figsize=(14, 6))
for i, seed in enumerate(SEED_LIST):
    success = np.array(training_results[seed]["best"]) >= 200
    rolling_success = pd.Series(success.astype(float)).rolling(50).mean() * 100
    plt.plot(rolling_success, color=seed_colors[i], linewidth=2, label=f"Seed {seed}")

plt.axhline(y=100, color='green', linestyle=':', alpha=0.4, label='100%')
plt.axhline(y=50, color='gray', linestyle=':', alpha=0.4, label='50%')
plt.title("GA Training: Rolling Success Rate - Best Genome (window=50)", fontsize=14)
plt.xlabel("Generation")
plt.ylabel("Success Rate (%)")
plt.ylim(-5, 105)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Eval Score Progression during Training (from periodic evaluations)

fig, axes = plt.subplots(1, len(SEED_LIST), figsize=(6 * len(SEED_LIST), 5), sharey=True)
if len(SEED_LIST) == 1:
    axes = [axes]

for ax, seed in zip(axes, SEED_LIST):
    history = eval_histories[seed]
    if history:
        gens = [e["generation"] for e in history]
        means = np.array([e["mean"] for e in history])
        stds = np.array([e["std"] for e in history])
        scores = [e["score"] for e in history]

        ax.plot(gens, means, color='blue', linewidth=2, label='Mean Reward')
        ax.fill_between(gens, means - stds, means + stds, color='blue', alpha=0.15)
        ax.plot(gens, scores, color='green', linewidth=1.5, linestyle='--', label='Score (mean-std)')
        ax.axhline(y=200, color='red', linestyle='--', alpha=0.5)
        ax.legend(fontsize=8)

    ax.set_title(f"Seed {seed}")
    ax.set_xlabel("Generation")
    ax.grid(True, alpha=0.3)

axes[0].set_ylabel("Reward")
fig.suptitle("GA - Deterministic Eval Score during Training", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Evaluation: deterministic episodes per seed (best genome)

evaluation_results = {}  # {seed: np.array}

for seed in SEED_LIST:
    print(f"Evaluating GA seed {seed} (best genome)...")

    set_all_seeds(seed)
    genome = best_genomes[seed]

    rewards = evaluate_genome_deterministic(genome, EVALUATION_EPISODES, seed=seed)
    evaluation_results[seed] = np.array(rewards)

    mean_r = np.mean(rewards)
    std_r = np.std(rewards)
    success = np.sum(np.array(rewards) >= 200) / len(rewards) * 100
    print(f"  Reward: {mean_r:.2f} +/- {std_r:.2f} | Success: {success:.0f}%")

print(f"\nEvaluation complete for all seeds.")

In [ ]:
# Evaluation Summary Table

rows = []
for seed in SEED_LIST:
    r = evaluation_results[seed]
    rows.append({
        "Seed": seed,
        "Mean Reward": f"{np.mean(r):.2f}",
        "Std Dev": f"{np.std(r):.2f}",
        "Min Reward": f"{np.min(r):.2f}",
        "Max Reward": f"{np.max(r):.2f}",
        "Success Rate": f"{(r >= 200).sum() / len(r) * 100:.1f}%"
    })

all_r = np.concatenate([evaluation_results[s] for s in SEED_LIST])
rows.append({
    "Seed": "Overall",
    "Mean Reward": f"{np.mean(all_r):.2f}",
    "Std Dev": f"{np.std(all_r):.2f}",
    "Min Reward": f"{np.min(all_r):.2f}",
    "Max Reward": f"{np.max(all_r):.2f}",
    "Success Rate": f"{(all_r >= 200).sum() / len(all_r) * 100:.1f}%"
})

print(f"*** GA MULTI-SEED EVALUATION SUMMARY ***")
print(f"Episodes per seed: {EVALUATION_EPISODES} | Total: {len(all_r)}")
print(pd.DataFrame(rows).to_string(index=False))

In [ ]:
# Evaluation Convergence Plots (per seed)

fig, axes = plt.subplots(1, len(SEED_LIST), figsize=(6 * len(SEED_LIST), 5), sharey=True)
if len(SEED_LIST) == 1:
    axes = [axes]

for ax, seed in zip(axes, SEED_LIST):
    rewards = evaluation_results[seed]
    episodes = np.arange(1, len(rewards) + 1)
    running_mean = np.cumsum(rewards) / episodes
    running_std = np.array([np.std(rewards[:i]) for i in episodes])

    ax.scatter(episodes, rewards, color='gray', alpha=0.4, s=20, label='Episode Reward')
    ax.plot(episodes, running_mean, color='blue', linewidth=2, label='Running Mean')
    ax.fill_between(episodes, running_mean - running_std, running_mean + running_std,
                    color='blue', alpha=0.15)
    ax.axhline(y=200, color='red', linestyle='--')
    ax.set_title(f"Seed {seed}")
    ax.set_xlabel("Episode")
    ax.grid(True, alpha=0.3)

axes[0].set_ylabel("Reward")
fig.suptitle(f"GA Evaluation: {EVALUATION_EPISODES} Episodes per Seed", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Evaluation Bar Chart (mean reward per seed with error bars)

all_r = np.concatenate([evaluation_results[s] for s in SEED_LIST])
means = [np.mean(evaluation_results[s]) for s in SEED_LIST]
stds_vals = [np.std(evaluation_results[s]) for s in SEED_LIST]
labels = [str(s) for s in SEED_LIST]

plt.figure(figsize=(max(8, 3 * len(SEED_LIST)), 6))
plt.bar(labels, means, yerr=stds_vals, capsize=5, color=seed_colors[:len(SEED_LIST)], alpha=0.8)
plt.axhline(y=200, color='red', linestyle='--', label='Solved Threshold (200)')
plt.axhline(y=float(np.mean(all_r)), color='blue', linestyle='-', linewidth=2,
            label=f'Overall Mean ({np.mean(all_r):.1f})')

plt.title(f"GA Mean Reward per Seed ({EVALUATION_EPISODES} episodes each)", fontsize=14)
plt.xlabel("Seed")
plt.ylabel("Mean Reward")
plt.legend()
plt.grid(True, alpha=0.3, axis='y')
plt.show()

In [ ]:
# Reward Distribution Histograms (overlaid per seed)

all_r = np.concatenate([evaluation_results[s] for s in SEED_LIST])

plt.figure(figsize=(12, 5))
for i, seed in enumerate(SEED_LIST):
    plt.hist(evaluation_results[seed], bins=10, alpha=0.5,
             color=seed_colors[i], edgecolor='black', label=f"Seed {seed}")

plt.axvline(x=float(np.mean(all_r)), color='blue', linestyle='-', linewidth=2,
            label=f'Overall Mean ({np.mean(all_r):.1f})')
plt.axvline(x=200, color='red', linestyle='--', label='Solved Threshold (200)')
plt.title('GA Reward Distribution across Seeds')
plt.xlabel('Reward')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Box Plot per Seed

fig, ax = plt.subplots(figsize=(8, 5))

data = [evaluation_results[seed] for seed in SEED_LIST]
bp = ax.boxplot(data, tick_labels=[str(s) for s in SEED_LIST], patch_artist=True)
for patch, color in zip(bp['boxes'], seed_colors[:len(SEED_LIST)]):
    patch.set_facecolor(color)
    patch.set_alpha(0.6)

ax.axhline(y=200, color='red', linestyle='--', label='Solved Threshold (200)')
ax.set_title(f"GA: Reward Distribution per Seed ({EVALUATION_EPISODES} episodes each)", fontsize=14)
ax.set_xlabel("Seed")
ax.set_ylabel("Reward")
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

---

## Baseline Comparison

In [ ]:
# Random Agent Baseline Evaluation

random_results = {}

for seed in SEED_LIST:
    print(f"Running random agent with seed {seed}...")
    env = gym.make(GYMNASIUM_MODEL, enable_wind=WIND_ENABLED)
    env.action_space.seed(seed)
    episode_rewards = []

    for ep in range(EVALUATION_EPISODES):
        obs, info = env.reset(seed=seed + ep)
        total_reward = 0.0
        done = False

        while not done:
            action = env.action_space.sample()
            obs, reward, terminated, truncated, info = env.step(action)
            total_reward += float(reward)
            done = terminated or truncated

        episode_rewards.append(total_reward)

    random_results[seed] = np.array(episode_rewards)
    env.close()

print("Random baseline evaluation complete.")

In [ ]:
# Baseline Comparison: Table + Chart

all_ga = np.concatenate([evaluation_results[s] for s in SEED_LIST])
all_random = np.concatenate([random_results[s] for s in SEED_LIST])

rows = [
    {
        "Agent": "Random",
        "Mean Reward": f"{np.mean(all_random):.2f}",
        "Std Dev": f"{np.std(all_random):.2f}",
        "Min": f"{np.min(all_random):.2f}",
        "Max": f"{np.max(all_random):.2f}",
        "Success Rate": f"{(all_random >= 200).sum() / len(all_random) * 100:.1f}%"
    },
    {
        "Agent": "GA",
        "Mean Reward": f"{np.mean(all_ga):.2f}",
        "Std Dev": f"{np.std(all_ga):.2f}",
        "Min": f"{np.min(all_ga):.2f}",
        "Max": f"{np.max(all_ga):.2f}",
        "Success Rate": f"{(all_ga >= 200).sum() / len(all_ga) * 100:.1f}%"
    },
]

print("*** BASELINE COMPARISON ***")
print(pd.DataFrame(rows).to_string(index=False))
print()

# Bar chart
agent_labels = ["Random", "GA"]
agent_means = [np.mean(all_random), np.mean(all_ga)]
agent_stds = [np.std(all_random), np.std(all_ga)]
bar_colors = ["gray", "tab:green"]

plt.figure(figsize=(8, 6))
bars = plt.bar(agent_labels, agent_means, yerr=agent_stds, capsize=6,
               color=bar_colors, alpha=0.8)
plt.axhline(y=200, color='red', linestyle='--', label='Solved Threshold (200)')

for bar, mean in zip(bars, agent_means):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 5,
             f'{mean:.1f}', ha='center', va='bottom', fontweight='bold')

plt.title(f"Baseline Comparison: Random vs GA ({EVALUATION_EPISODES * len(SEED_LIST)} episodes each)", fontsize=14)
plt.ylabel("Mean Reward")
plt.legend()
plt.grid(True, alpha=0.3, axis='y')
plt.show()

---

## Statistical Significance

In [ ]:
# Statistical Significance: GA vs Random (Mann-Whitney U + Chi-squared)

# --- Reward comparison (Mann-Whitney U) ---
mwu_result = stats.mannwhitneyu(all_ga, all_random, alternative='two-sided')
stat_reward = float(mwu_result.statistic)
p_reward = float(mwu_result.pvalue)

# --- Success rate comparison (Chi-squared) ---
ga_successes = int((all_ga >= 200).sum())
random_successes = int((all_random >= 200).sum())
ga_total = len(all_ga)
random_total = len(all_random)

contingency = np.array([
    [ga_successes, ga_total - ga_successes],
    [random_successes, random_total - random_successes]
])

if np.all(contingency.sum(axis=0) > 0) and np.all(contingency.sum(axis=1) > 0):
    result = stats.chi2_contingency(contingency)
    chi2 = float(result.statistic)  # type: ignore[attr-defined]
    p_success = float(result.pvalue)  # type: ignore[attr-defined]
    chi2_valid = True
else:
    chi2, p_success = 0.0, 1.0
    chi2_valid = False

# --- Results table ---
chi2_note = "" if chi2_valid else " (skipped: zero row/col)"
rows = [
    {
        "Metric": "Mean Reward",
        "GA Value": f"{np.mean(all_ga):.2f}",
        "Random Value": f"{np.mean(all_random):.2f}",
        "Test": "Mann-Whitney U",
        "Statistic": f"{stat_reward:.1f}",
        "p-value": f"{p_reward:.4f}",
        "Significant (p<0.05)": "Yes" if p_reward < 0.05 else "No"
    },
    {
        "Metric": "Success Rate (>=200)",
        "GA Value": f"{ga_successes/ga_total*100:.1f}%",
        "Random Value": f"{random_successes/random_total*100:.1f}%",
        "Test": f"Chi-squared{chi2_note}",
        "Statistic": f"{chi2:.2f}",
        "p-value": f"{p_success:.4f}",
        "Significant (p<0.05)": "Yes" if (chi2_valid and p_success < 0.05) else "No"
    },
]

print("*** STATISTICAL SIGNIFICANCE TESTS: GA vs Random ***")
print(f"Sample size per agent: {ga_total} episodes ({EVALUATION_EPISODES} episodes x {len(SEED_LIST)} seeds)")
print()
print(pd.DataFrame(rows).to_string(index=False))
print()
if p_reward < 0.05:
    print(f"The reward difference between GA and Random is statistically significant (p={p_reward:.4f}).")
else:
    print(f"No statistically significant reward difference between GA and Random (p={p_reward:.4f}).")

---

## Agent Behavior Analysis

In [ ]:
# Collect per-step data: actions and trajectories

action_counts = np.zeros(len(ACTION_LABELS), dtype=int)
trajectory_data = []  # list of (x_positions, y_positions)

for seed in SEED_LIST:
    genome = best_genomes[seed]
    nn = NeuralNetwork(INPUT_SIZE, HIDDEN1_SIZE, HIDDEN2_SIZE, OUTPUT_SIZE, genome)
    env = gym.make(GYMNASIUM_MODEL, enable_wind=WIND_ENABLED)

    for ep in range(EVALUATION_EPISODES):
        obs, info = env.reset(seed=seed + ep)
        done = False
        x_pos, y_pos = [obs[0]], [obs[1]]

        while not done:
            output = nn.forward(obs)
            action = int(np.argmax(output))
            action_counts[action] += 1

            obs, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            x_pos.append(obs[0])
            y_pos.append(obs[1])

        # Keep trajectory for the first TRAJECTORY_EPISODES episodes of the first seed
        if seed == SEED_LIST[0] and ep < TRAJECTORY_EPISODES:
            trajectory_data.append((np.array(x_pos), np.array(y_pos)))

    env.close()

total_actions = action_counts.sum()
print(f"GA: {total_actions:,} total actions collected across {EVALUATION_EPISODES * len(SEED_LIST)} episodes")
print("\nBehavior data collection complete.")

In [ ]:
# Action Distribution

n_actions = len(ACTION_LABELS)
x = np.arange(n_actions)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Absolute counts
ax1.bar(x, action_counts, color='tab:green', alpha=0.8)
ax1.set_xticks(x)
ax1.set_xticklabels(ACTION_LABELS, rotation=15)
ax1.set_title("Action Counts (Absolute)", fontsize=13)
ax1.set_ylabel("Count")
ax1.grid(True, alpha=0.3, axis='y')

# Percentage distribution
pcts = action_counts / action_counts.sum() * 100
bars = ax2.bar(x, pcts, color='tab:green', alpha=0.8)
for bar, pct in zip(bars, pcts):
    ax2.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.5,
             f'{pct:.1f}%', ha='center', va='bottom', fontsize=9)

ax2.set_xticks(x)
ax2.set_xticklabels(ACTION_LABELS, rotation=15)
ax2.set_title("Action Distribution (Percentage)", fontsize=13)
ax2.set_ylabel("Percentage (%)")
ax2.grid(True, alpha=0.3, axis='y')

fig.suptitle(f"GA: Action Distribution ({EVALUATION_EPISODES * len(SEED_LIST)} episodes)", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Trajectory Plots: x-y paths of the lander

traj_colors = list(plt.colormaps["Set2"](range(8)))

fig, ax = plt.subplots(figsize=(10, 7))

for i, (x_pos, y_pos) in enumerate(trajectory_data):
    ax.plot(x_pos, y_pos, color=traj_colors[i], linewidth=1.5, alpha=0.8,
            label=f"Episode {i+1}")
    ax.scatter(x_pos[0], y_pos[0], color=traj_colors[i], marker='o', s=60, zorder=5)
    ax.scatter(x_pos[-1], y_pos[-1], color=traj_colors[i], marker='x', s=80, zorder=5)

ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5, alpha=0.3)
ax.axvline(x=0, color='black', linestyle='-', linewidth=0.5, alpha=0.3)
ax.scatter(0, 0, color='red', marker='^', s=120, zorder=10, label='Landing Pad')

ax.set_title(f"GA Lander Trajectories (seed {SEED_LIST[0]}, {TRAJECTORY_EPISODES} episodes)", fontsize=14)
ax.set_xlabel("X Position")
ax.set_ylabel("Y Position")
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)
plt.show()

---

## GIF Visualizations

In [ ]:
# GIF Visualization (one per seed, best genome)

output_dir = os.path.join(NOTEBOOK_DIR, "outputs_ga")
os.makedirs(output_dir, exist_ok=True)

for seed in SEED_LIST:
    print(f"Generating GIF for GA seed {seed} (best genome)...")

    gif_path = os.path.join(output_dir, f"ga_seed{seed}.gif")
    record_genome_gif(best_genomes[seed], seed, gif_path)

    print(f"  Saved: {gif_path}")
    display(Image(filename=gif_path))

In [ ]:
# Hyperparameter Table

params = {
    "input_size": INPUT_SIZE,
    "hidden1_size": HIDDEN1_SIZE,
    "hidden2_size": HIDDEN2_SIZE,
    "output_size": OUTPUT_SIZE,
    "population_size": POPULATION_SIZE,
    "mutation_rate": MUTATION_RATE,
    "generations": GENERATIONS,
    "eval_seeds_per_generation": EVAL_SEEDS_PER_GEN,
    "elitism": 3,
    "parent_selection": "top 20%",
    "crossover": "uniform gene-wise",
    "mutation": "Gaussian (clipped +/-0.1)",
    "activation": "tanh (hidden), linear (output)",
    "max_workers": MAX_WORKERS,
}

rows = [{"Parameter": k, "Value": str(v)} for k, v in params.items()]

print("*** GA Hyperparameters ***")
print(pd.DataFrame(rows).to_string(index=False))

---

## Appendix: Experimental Setup

### Environment Details

| Property | Value |
|---|---|
| Environment | `LunarLander-v3` (Gymnasium) |
| Observation Space | `Box(8,)` — continuous 8-dimensional vector |
| Action Space | `Discrete(4)` — do nothing, fire left, fire main, fire right |
| Solved Threshold | Mean reward >= 200 over 100 consecutive episodes |
| Wind | Disabled (`enable_wind=False`) |

**Observation vector:** `[x, y, vx, vy, angle, angular_velocity, left_leg_contact, right_leg_contact]`

**Reward structure:**
- Moving toward the landing pad: positive
- Moving away: negative
- Crash: -100
- Successful landing: +100
- Each leg ground contact: +10
- Firing main engine: -0.3 per frame
- Firing side engine: -0.03 per frame

**Termination rules:**
- **Terminated (success):** The lander comes to rest on the ground with both legs in contact, near-zero velocity
- **Terminated (crash):** The lander body contacts the ground, or the lander moves outside the viewport boundaries
- **Truncated (timeout):** The episode exceeds 1000 timesteps without termination

In [ ]:
# System and library versions

print(f"Python: {sys.version.split()[0]}")
print(f"Gymnasium: {gym.__version__}")
print(f"NumPy: {np.__version__}")
print(f"Network: {INPUT_SIZE} -> {HIDDEN1_SIZE} -> {HIDDEN2_SIZE} -> {OUTPUT_SIZE}")
print(f"Activation: tanh (hidden), linear (output)")

---